# INTERMEDIATE  MACHINE LEARNING

## Table of Contents
- [Missing Values](#missing-values)
- [Categorical Variables](#categorical-variables)

### Missing Values

A la hora de modelar nuestra data, puede haber casos en los que nos encontremos con valores vacíos y muchas de las librerías (como *scikit-learn*) de Machine Learning darán error a la hora de intentar construir el modelo. Para lidiar con ello, elegiremos algunas de las siguientes estrategias:

* **Eliminar las columnas con los valores perdidos**: Con esta técnica, nos arriesgamos a perder información potencialmente útil en el modelo. Si hay 10k filas, y solo 1 está vacía, perderíamos toda la columna solo por ese valor.

* **Imputación**: Consiste en rellenar los valores vacíos con algún dato aleatorio. El valor generalmente no será el correcto, pero nos dará un modelo más preciso que eliminando toda la información de la columna. 

* **Extensión de imputación**: En este caso, hacemos una imputación, pero también registramos en una nueva columna si los valores originales de la columna estaban vacíos o no. En algunos casos, ayudará a hacer el modelo más preciso, y en otros casos no ayudará en nada. 

Empezaremos viendo un ejemplo con el dataset que tenemos. Predeciremos el precio, y para simplificar, solo usaremos predictores numéricos

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the data
data = pd.read_csv('melb_data.csv')

# Select target
y = data.Price

# To keep things simple, we'll use only numerical predictors
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

Vamos a crear una función para comparar las diferentes métodos que tenemos al trabajar con valores vacíos. Para ello estudiaremos el **_Mae_** de un modelo Random Forest

In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

Comparemos las diferentes opciones que hemos visto antes para tratar con valores vacíos


**Método 1: Eliminar las columnas**

Tenemos que tener cuidado a la hora de seleccionar la data para hacer el modelo y para testear de que en ambos sets eliminemos las mismas columnas 

In [18]:
# Get names of columns with missing values
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]

# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE from Approach 1 (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop columns with missing values):
183550.22137772635


**Método 2: Imputación**

Para la imputación, vamos a usar el método SimpleImputer de la librería *sklearn* para sustituir los valores vacíos con el valor medio de cada columna. 

In [19]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from Approach 2 (Imputation):
178166.46269899711


Como podemos observar, el método 2 nos ha dado un MAE menor que el método 1, por lo que sería una mejor aproximación.

**Método 3: Extensión de imputación**

Vamos a imputar ahora los valores vacíos mientras registramos que valores han sido imputados.

In [20]:
# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

MAE from Approach 3 (An Extension to Imputation):
178927.503183954


Observamos que el MAE de este método es ligeramente peor que el del método 2, por lo que no nos proporciona ninguna mejora significativa registrar que valores han sido cambiados.

**Conclusión**

Hemos podido observar que imputar los valores vacíos es mejor que eliminar las columnas con valores vacíos. Esto se debe a que en este modelo, solo 3 columnas del dataset tienen valores vacíos, y de esas 3 columnas, todas tienen menos de la mitad de filas vacías, por lo que al eliminarlas, perderíamos más de la mitad de la información que nos podría ser significativa. Puede que en otros modelos se dé otro caso donde eliminar las columnas de mejor resultado.

In [21]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = X_train.isnull().sum()

# Filter columns of the serie which value is higher than 0
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(10864, 12)
Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64


### Categorical Variables

Decimos que una variable es **Categórica** cuando los valores que almacenan se pueden categorizar, como por ejemplo marcas de coches o frecuencias (siempre, a veces, nunca). Normalmente necesitaremos pre procesar estas variables antes de usarlas en nuestro modelo para que no tengamos errores. Vamos a ver 3 métodos que podemos usar.

- **Drop Categorical Variables**: La forma más simple de tratar con ellas es eliminarlas del modelo, pero esto solo funcionará si son columnas sin información útil.

- **Ordinal Encoding**: La codificación ordinal asigna cada valor único de la categoría a un número. En casos de que haya un orden en las categorías, asignaremos los valores según dicho orden. Estas variables se les llama **variables ordenadas**.

<img src="../Intro%20to%20Machine%20Learning/Images/Ordinal%20Encoding.png" alt="drawing" width="750"/>


- **One-Hot Encoding**: La codificación One-hot crea una nueva columna que indica la presencia o ausencia de cada variable en la data. A diferencia del Ordinal Encoding, la codificación One-hot no asume que las categorías estén ordenadas. A estas variables categóricas sin orden especifico (como por ejemplo los colores) se les llama **variables nominales**. Este método no suele ser muy útil si la variable categórica tiene muchos valores.

<img src="../Intro%20to%20Machine%20Learning/Images/One-hot%20Encoding.png" alt="drawing" width="750"/>



Vamos a ver en un ejemplo como trabajar con estas variables categóricas. Empezamos seteando las variables y eliminando las variables con valores vacíos

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('melb_data.csv')

# Separate target from predictors
y = data.Price
X = data.drop(['Price'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# Drop columns with missing values (simplest approach)
cols_with_missing = [col for col in X_train_full.columns if X_train_full[col].isnull().any()] 
X_train_full.drop(cols_with_missing, axis=1, inplace=True)
X_valid_full.drop(cols_with_missing, axis=1, inplace=True)

Ahora nos quedaremos con las columnas con variables categóricas que tienen pocas categorías y con las numéricas (ya que son las que nos interesan de verdad).

In [23]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.8272,4217.0


Podemos ver las variables categóricas que son Type, Method y Regionname, pero la forma de obtenerlas (en caso de que nuestro dataset sea enorme) seria viendo que columnas tienen tipo object (texto) 

In [24]:
# Get list of categorical variables
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

Categorical variables:
['Type', 'Method', 'Regionname']


Vamos a ver ahora como de buenos son los diferentes métodos que hemos visto. Para ello usaremos la función que creamos anterior *score_dataset()*

**Método 1: Eliminar las columnas**

Vamos a eliminar todas las columnas que no sean de tipo numérico, es decir, las columnas de tipo *object* de los datasets de entreno y de validación. 

In [25]:
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop categorical variables):


183550.22137772635


**Método 2: Ordinal Encoding**

El módulo *scikit-learn* tiene una clase **OrdinalEncoder** que se puede usar directamente para obtener el ordinal encodings en el dataset. Hacemos un bucle en cada columna con variable categórica para obtener los códigos ordinales por separados.

In [26]:
from sklearn.preprocessing import OrdinalEncoder

# Make copy to avoid changing original data 
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Apply ordinal encoder to each column with categorical data
ordinal_encoder = OrdinalEncoder()
label_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
label_X_valid[object_cols] = ordinal_encoder.transform(X_valid[object_cols])

print("MAE from Approach 2 (Ordinal Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE from Approach 2 (Ordinal Encoding):
175062.2967599411


**Método 3: One-Hot Encoding**

En este método también usaremos la clase **OneHotEncoder** del módulo *scikit-learn*. Podemos usar algunos parámetros para personalizar el comportamiento de la clase:
- *handle_unknown=\'ignore\'* para evitar errores cuando la data de validación contenga clases que no están representadas en la data de entrenamiento
- *sparse_output=False* nos asegura que las columnas codificadas se devuelvan como un numpy array en vez de una matriz

In [27]:
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

# Ensure all columns have string type
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)

print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE from Approach 3 (One-Hot Encoding):
176703.63810751104


**Conclusión**

Como hemos visto, dropear las columnas ha sido la peor aproximación. Entre los otros dos métodos, los MAE ha sido valores muy parecidos, por lo que no habrá mucha diferencia en usar un método o el otro. En general el one-hot encoding será la opción que mejor funcione, y dropear las columnas la que peor funcione.